Get OpenAI key from:

[xebia-openai-us](https://portal.azure.com/#@xebia.com/resource/subscriptions/5ddf05c0-b972-44ca-b90a-3e49b5de80dd/resourceGroups/xebia-openai-us/providers/Microsoft.CognitiveServices/accounts/xebia-openai-us/cskeys)

<br>

Set in `.env` file:
```
OPENAI_API_TYPE="azure"
OPENAI_API_BASE="https://xebia-openai-us.openai.azure.com"
OPENAI_API_VERSION="2023-03-15-preview"
OPENAI_API_KEY="<PASTE KEY HERE>"
```

Then export using: `source .env`

In [63]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
from crea_scraper.search import get_relevant_courses
from crea_scraper.data import get_course_documents_for_search, load_course_data

import os
from typing import List

from langchain.llms import AzureOpenAI
from langchain.schema import Document
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

from crea_scraper.prompts import search_prompt_template
from crea_scraper.vector_db import load_vector_db

In [74]:
lmm = AzureOpenAI(
    model_name="gpt-4",
    temperature=0.7,
    max_tokens=250,
    top_p=1.0,
    verbose=True,
    engine="gpt-4-us",
)

In [76]:
lmm("hi")

'Hello! How can I help you today?'

In [55]:
course_data = load_course_data("../output/course_data.csv")
course_documents = get_course_documents_for_search(course_data)

In [56]:
docs = get_relevant_courses(
    "Ik wil iets doen met hout.",
    course_documents,
    method="embedding",
    k=15,
    vector_db_path="../output/vector_db",
    verbose=True
)

Search query:

hout, houtbewerking, woodworking, houtsnijden, sawing, meubels maken, furniture making, schaven, sanding, boren, drilling, zagen, cutting, lijmen, gluing.


In [57]:
[dc.metadata["naam"] for d in docs]

['Beeldhouwen in hout',
 'Beeldhouwtechnieken',
 'Stilleven; werken met diverse materialen',
 'Boetseren',
 'Toegepaste keramiek',
 'Tekenen',
 'Schilderen',
 'Tekenen en boetseren',
 'Modeltekenen- en schilderen',
 'Modelboetseren',
 'Tekenen en schilderen',
 'Beelden van keramiek',
 'Graphic Arts',
 'Ets- en andere druktechnieken',
 'Creatief schrijven']

In [60]:
docs

[Document(page_content='Course title: Beeldhouwen in hout\nCourse description: Deze korte cursus is een kennismaking met de techniek van het hakken en snijden in hout.Hout is een mooi materiaal met een warme uitstraling.\n\nWat doe je?Je maakt een klein beeldje in Lindehout of Abachi en al doende leer je het hout kennen en het werken met gutsen en een klopper. Om in deze cursus zo snel mogelijk te beginnen met hout wordt er een stilleven neergezet dat je op jouw manier gaat verbeelden. Je leert kijken naar onderlinge verhoudingen; ritme en de ruimtelijke werking. Het hakken en snijden in hout vraagt enige oefening; maar dat kun je snel onder de knie krijgen.\n\nDiegenen die in het vorige blok met de cursus zijn begonnen zijn welkom om in deze cursus verder te werken aan hun beeldje.\n\nInloopatelier ModeltekenenWil je meer? Elke zaterdagochtend is er van 10.30-13.00 uur inloopatelier modeltekenen met (naakt)model onder begeleiding van CREA-docenten Beeldend.', metadata={'naam': 'Beeldh